In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('SWaT.csv')
df.columns = df.columns.str.strip()

In [3]:
df['Normal/Attack'] = (df['Normal/Attack'] == 'Attack').astype(int)
df = df.drop(['Timestamp'], axis=1)     # If timestamp is unnecessary
# Remove constant columns if needed:
df = df.loc[:, df.std() > 0]

In [4]:
from sklearn.preprocessing import MinMaxScaler
X = df.drop('Normal/Attack', axis=1)
y = df['Normal/Attack']

scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42
)

In [6]:
# Example: correlation on train
corr_matrix = X_train.corrwith(y_train).abs()
top_10_corr = corr_matrix.nlargest(10).index
X_train_selected = X_train[top_10_corr]
X_test_selected  = X_test[top_10_corr]

In [7]:
from reservoirpy import ESN
y_train_arr = np.array(y_train).reshape(-1, 1)
esn = ESN(units=10, sr=0.1, input_scaling=1.0, ridge=1e-6, seed=42)
esn = esn.fit(X_train_selected.values, y_train_arr)
y_pred_test = esn.run(X_test_selected.values)
from sklearn.metrics import accuracy_score, f1_score
acc = accuracy_score(y_test.values, (y_pred_test > 0.5).astype(int).ravel())
f1 = f1_score(y_test.values, (y_pred_test > 0.5).astype(int).ravel(), average='weighted')
print(f"Accuracy: {acc:.4f} | F1: {f1:.4f}")

Accuracy: 0.9531 | F1: 0.9482
